Convert Twitter JSON files to a pandas dataframe.

In [ ]:
import pandas
import json
from datetime import datetime
import bs4

user_info = [
    "name",
    "screen_name",
    "description",
    "verified",
    "followers_count",
    "friends_count",
    "created_at",
    "location",
    "time_zone",
    "statuses_count",
    "favourites_count"
]

user_prefix = "user_"

user_info_columns = [user_prefix + x for x in user_info]

export_columns = [
    "created_at",
    "account_age",
    "source", 
    "truncated",
    "in_reply_to_status_id",
    "in_reply_to_user_id",
    "lang",
    "coordinates",
    "tweet_url",
    "text"] +\
    user_info_columns + [
    "hashtags",
    "hashtags_count",
    "mentions",
    "mentions_count",
    "urls",
    "urls_count",
    "media_urls",
    "media_type_count",
    "media_type"
]

def to_dict(tweet):
    try:
        tweet = json.loads(tweet)
        tweet_id = tweet["id"] 
        del tweet["id"] 
        return (tweet_id, tweet)
    except:
        return None

def extract_user_info(user): 
    return pandas.Series({user_prefix + i: user[i] for i in user_info})

def extract_attribute(obj, attr):
    def extract(entity):
        return [o[attr] for o in entity[obj]]
    return extract

def extract_extended_entities(entity):
    result = pandas.Series({"media_urls": [], "media_type": []})
    if not pandas.isnull(entity):
        for item in entity["media"]:
            result.media_urls.append(item["display_url"])
            result.media_type.append(item["type"])
    return result

def extract_source(source):
    if bs4.BeautifulSoup(source, 'html.parser').a is None: return None
    return bs4.BeautifulSoup(source, 'html.parser').a.text

def make_counts(df, column):
    df[column + "_count"] = df[column].map(lambda l: len(l))

def fix_text(string):
    if string is None: return None
    return string.replace("\n", " ").replace("\r", " ").replace("&amp;", "&")

def add_tweet_url(row):
    return "https://www.twitter.com/" + row.user_screen_name + "/status/" + str(row.name)

def json_to_df(filename):
    tweets_file = open(filename, "r")
    tweets = tweets_file.readlines()
    tweets_file.close()

    filtered_tweets = filter(lambda s: s.startswith('{"created_at":'), tweets)

    tweets_dict = map(to_dict, filtered_tweets) # Will contain Nones.

    df = pandas.DataFrame.from_dict({k: v for (k, v) in filter(lambda s: s!=None, tweets_dict)}).transpose()

    #errors = len(list(filter(lambda s: s==None, tweets_dict))) # Returns 0 for some reason!

    df = pandas.DataFrame.join(df, df.user.apply(extract_user_info))

    df["hashtags"] = df.entities.apply(extract_attribute("hashtags", "text")).apply(lambda tags: [tag.lower() for tag in tags])
    df["mentions"] = df.entities.apply(extract_attribute("user_mentions", "screen_name"))
    df["urls"] = df.entities.apply(extract_attribute("urls", "url"))

    df = pandas.DataFrame.join(df, df.extended_entities.apply(extract_extended_entities))

    make_counts(df, "hashtags")
    make_counts(df, "mentions")
    make_counts(df, "urls")
    make_counts(df, "media_type")

    df.text = df.text.apply(fix_text)
    df.user_description = df.user_description.apply(fix_text)

    df["tweet_url"] = df.apply(add_tweet_url, axis="columns")

    df.created_at = df.created_at.apply(pandas.to_datetime)
    df[user_prefix + "created_at"] = df[user_prefix + "created_at"].apply(pandas.to_datetime)
    df["account_age"] = df.created_at - df.user_created_at
    df["account_age"] = df.account_age.apply(lambda x: '{:0.2f}'.format(x.days / 365))
    df[user_prefix + "created_at"] = df[user_prefix + "created_at"].apply(lambda x: '{:%Y-%m-%d}'.format(datetime(x.year, x.month, x.day)))
    df["source"] = df.source.apply(extract_source)

    final_df = df[export_columns]

    return final_df
    #return final_df, errors